In [1]:
import pandas as pd

# 섹션 분리
- 2, 3만 사용

In [77]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    # 함음됨임봄짐움 으로 끝나고, (공백과 개행문자 혹은 . 혹은 ;로 끝난다) 혹은 ('다'로 끝나고 .) 공백)인 경우
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [78]:
import re 

def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section) #####

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section) #####
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section) #####

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section) #####

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section) #####

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section) #####

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)


    return sections, section_texts

In [125]:
df = pd.read_csv('c:/Users/user/MPB_minutes/MPB_minutes.csv')

In [126]:
zero_list = []
for idx,i in enumerate(df.content):
    if len(preprocess_minutes(df.content[idx])[1][0]) + len(preprocess_minutes(df.content[idx])[1][1]) +\
        len(preprocess_minutes(df.content[idx])[1][2]) + len(preprocess_minutes(df.content[idx])[1][3]) +\
        len(preprocess_minutes(df.content[idx])[1][4]) + len(preprocess_minutes(df.content[idx])[1][5]) == 0:
        zero_list.append(idx)

In [127]:
zero_title_list = []
for i in zero_list:
    zero_title_list.append(df.title[i])

In [128]:
zero_title_list[:10] # preprocess_minutes함수 적용 결과 빈 리스트가 리턴되는 경우, 위 6개의 카테고리의 시작부분에 해당하는 내용을 언급하지 않았다.
# ex) 금융시장 동향과 관련하여~

['금융통화위원회 의사록(2008년도 제11차)',
 '금융통화위원회 의사록(2008년도 제13차)',
 '금융통화위원회 의사록(2008년도 제15차)',
 '금융통화위원회 의사록(2008년도 제19차)',
 '제21차 금통위 의사록 (8)',
 '제22차 금통위 의사록 (6)',
 '제25차 금통위 의사록 (7)',
 '제27차 금통위 의사록 (2)',
 '제1차 금통위 의사록 (9)',
 '제3차 금통위 의사록 (8)']

### df의 content 열을 2,3 섹션만 남기기

In [129]:
df['processed_content'] = 0

for idx in range(df.shape[0]):
    df.processed_content[idx] = preprocess_minutes(df.content[idx])[1]
df

C:\Users\user\anaconda3\envs\mulcam\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\user\anaconda3\envs\mulcam\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,date,title,content,processed_content
0,2008-04-10,금융통화위원회 의사록(2008년도 제8차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"([일부 위원은 금년 하반기중 국내경제가 소,｢ ｣ 프트패치 로 평가된 년 분기부터..."
1,2008-05-08,금융통화위원회 의사록(2008년도 제10차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"([일부 위원은 최근 경기상승세가 둔화되고,｢ ｣ 있는 가운데 일각에서는 경기가 금..."
2,2008-05-22,금융통화위원회 의사록(2008년도 제11차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"([], [], [], [], [], [])"
3,2008-06-12,금융통화위원회 의사록(2008년도 제12차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"([일부 위원은 금년 월중 제조업 및 서비, 4 5｢ ｣ ～ 스업 생산활동이 비교적..."
4,2008-06-26,금융통화위원회 의사록(2008년도 제13차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"([], [], [], [], [], [])"
...,...,...,...,...
250,2020-06-11,제13차_금통위 의사록,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"([], [], [], [], [], [])"
251,2020-06-24,제14차_금통위 의사록,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"([], [], [], [], [], [])"
252,2020-07-16,제15차_금통위 의사록,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,([일부 위원은 지난 경제전망 이후 민간소비의 회복 속도가 예상보다 더딘 데 다 ...
253,2020-07-17,제16차_금통위 의사록,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"([], [], [], [], [], [])"


In [130]:
df = df.drop('content', axis=1).rename(columns={'processed_content':'content'})

In [131]:
df

,date,title,content
0,2008-04-10,금융통화위원회 의사록(2008년도 제8차),"([일부 위원은 금년 하반기중 국내경제가 소,｢ ｣ 프트패치 로 평가된 년 분기부터..."
1,2008-05-08,금융통화위원회 의사록(2008년도 제10차),"([일부 위원은 최근 경기상승세가 둔화되고,｢ ｣ 있는 가운데 일각에서는 경기가 금..."
2,2008-05-22,금융통화위원회 의사록(2008년도 제11차),"([], [], [], [], [], [])"
3,2008-06-12,금융통화위원회 의사록(2008년도 제12차),"([일부 위원은 금년 월중 제조업 및 서비, 4 5｢ ｣ ～ 스업 생산활동이 비교적..."
4,2008-06-26,금융통화위원회 의사록(2008년도 제13차),"([], [], [], [], [], [])"
...,...,...,...
250,2020-06-11,제13차_금통위 의사록,"([], [], [], [], [], [])"
251,2020-06-24,제14차_금통위 의사록,"([], [], [], [], [], [])"
252,2020-07-16,제15차_금통위 의사록,([일부 위원은 지난 경제전망 이후 민간소비의 회복 속도가 예상보다 더딘 데 다 ...
253,2020-07-17,제16차_금통위 의사록,"([], [], [], [], [], [])"


## ngram 만들기

In [132]:
from ekonlpy.tag import Mecab
from ekonlpy.sentiment import MPCK
from tqdm.notebook import tqdm
import pandas as pd

In [133]:
# content column을 flatten한다.

for i in range(df.shape[0]):
    df.content[i] = list(sum(df.content[i], []))

In [134]:
df['ngram'] = 0
df

,date,title,content,ngram
0,2008-04-10,금융통화위원회 의사록(2008년도 제8차),"[일부 위원은 금년 하반기중 국내경제가 소,｢ ｣ 프트패치 로 평가된 년 분기부터 ...",0
1,2008-05-08,금융통화위원회 의사록(2008년도 제10차),"[일부 위원은 최근 경기상승세가 둔화되고,｢ ｣ 있는 가운데 일각에서는 경기가 금년...",0
2,2008-05-22,금융통화위원회 의사록(2008년도 제11차),[],0
3,2008-06-12,금융통화위원회 의사록(2008년도 제12차),"[일부 위원은 금년 월중 제조업 및 서비, 4 5｢ ｣ ～ 스업 생산활동이 비교적 ...",0
4,2008-06-26,금융통화위원회 의사록(2008년도 제13차),[],0
...,...,...,...,...
250,2020-06-11,제13차_금통위 의사록,[],0
251,2020-06-24,제14차_금통위 의사록,[],0
252,2020-07-16,제15차_금통위 의사록,[일부 위원은 지난 경제전망 이후 민간소비의 회복 속도가 예상보다 더딘 데 다 수...,0
253,2020-07-17,제16차_금통위 의사록,[],0


### N-grams Feature

In [135]:
mpck = MPCK()

for idx_sentences, sentences in tqdm(enumerate(df.content)):
    token_list = []
    for idx_sentence, sentence in enumerate(sentences):
        token = mpck.tokenize(df.content[idx_sentences][idx_sentence])
        token_list.append(token)
    tokens = list(set(sum(token_list,[])))
    ngrams = mpck.ngramize(tokens)
    df['ngram'].loc[idx_sentences] = ngrams+tokens

C:\Users\user\anaconda3\envs\mulcam\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [141]:
len(df['ngram'][3])

904

In [125]:
call = pd.read_csv('c:\\Users\\user\\call_rate_finish.csv')
df.date = pd.to_datetime(df.date)
call.date = pd.to_datetime(call.date) # df & call의 date 형식을 맞춘다.
call

,date,callrate,month_date,month_rate,label
0,2004-11-29,3.25,2004.10.29,0.00,up
1,2004-11-30,3.26,2004.10.30,0.00,up
2,2004-12-01,3.26,2004.11.01,0.00,up
3,2004-12-02,3.26,2004.11.02,0.00,up
4,2004-12-03,3.26,2004.11.03,0.00,up
...,...,...,...,...,...
5753,2020-08-30,0.49,2020.07.30,0.49,same
5754,2020-08-31,0.55,2020.07.31,0.51,up
5755,2020-09-01,0.52,2020.08.01,0.51,up
5756,2020-09-02,0.49,2020.08.02,0.51,down


In [126]:
df_callrate = pd.merge(df, call[['date','label']], on='date')
df_callrate

,date,title,content,ngram,token,label
0,2008-04-10,금융통화위원회 의사록(2008년도 제8차),"일부 위원은 금년 하반기중 국내경제가 소,｢ ｣ 프트패치 로 평가된 년 분기부터 년...","[국제/NNG;금융시장/NNG;불안/NNG;경기/NNG;부진/NNG, 국제/NNG;...","[중/NNG, 경제/NNG, 소/NNG, 트/NNG, 패치/NNG, 평가/NNG, ...",down
1,2008-05-08,금융통화위원회 의사록(2008년도 제10차),"일부 위원은 최근 경기상승세가 둔화되고,｢ ｣ 있는 가운데 일각에서는 경기가 금년 ...","[국제/NNG;금융시장/NNG;불안/NNG;경기/NNG;부진/NNG, 물가/NNG;...","[경기/NNG, 상승/NNG, 둔화/NNG, 되/XSV, 있/VA, 가운데/NNG,...",down
2,2008-05-22,금융통화위원회 의사록(2008년도 제11차),,[],[],up
3,2008-06-12,금융통화위원회 의사록(2008년도 제12차),"일부 위원은 금년 월중 제조업 및 서비, 4 5｢ ｣ ～ 스업 생산활동이 비교적 괜...","[국제/NNG;금융시장/NNG;불안/NNG;경기/NNG;부진/NNG, 국제/NNG;...","[월중/NNG, 제조업/NNG, 업/NNG, 생산/NNG, 활동/NNG, 비교적/V...",down
4,2008-06-26,금융통화위원회 의사록(2008년도 제13차),,[],[],up
...,...,...,...,...,...,...
250,2020-06-11,제13차_금통위 의사록,,[],[],down
251,2020-06-24,제14차_금통위 의사록,,[],[],down
252,2020-07-16,제15차_금통위 의사록,일부 위원은 지난 경제전망 이후 민간소비의 회복 속도가 예상보다 더딘 데 다 수출...,"[국고채/NNG;금리/NNG;하락/NNG;금리/NNG;인하/NNG, 성장/NNG;경...","[경제/NNG, 전망/NNG, 민간/NNG, 소비/NNG, 회복/NNG, 속도/NN...",up
253,2020-07-17,제16차_금통위 의사록,,[],[],down


In [129]:
df_callrate.to_csv('c:/Users/user/df_label.csv')